# AutoML

This is a component that performs predictions using an AutoML implementation: [autosklearn](https://github.com/automl/auto-sklearn). 
<br>
auto-sklearn is an automated machine learning toolkit and a drop-in replacement for a scikit-learn estimator.

This notebook shows:
- how to use SDK to load a model.
- how to use a model to provide real-time predictions.

In [ ]:
%%writefile Predictor.py
import logging
from typing import List, Iterable, Union

import numpy as np
import pandas as pd
from platiagro import load_model, load_metadata, load_dataset
from platiagro.featuretypes import CATEGORICAL, NUMERICAL, infer_featuretypes

logger = logging.getLogger(__name__)


class Predictor(object):
    """
    Model template. You can load your model parameters in __init__ from a location accessible at runtime.
    """

    def __init__(self, dataset=None, target=None, experiment_id=None):
        """
        Add any initialization parameters. These will be passed at runtime from the graph definition parameters defined in your seldondeployment kubernetes resource manifest.
        """
        logger.info("Initializing")

        # Loads Estimator and Label Encoders
        model = load_model(name=experiment_id)
        self.estimator = model["estimator"]
        self.les = model["label_encoder"]

        # Loads Column Names and Feature Types
        df = load_dataset(name=dataset)
        self.columns = df.columns.values.tolist()
        self.featuretypes = infer_featuretypes(df)

        # Finds the Feature Type for target variable 
        target_idx = self.columns.index(target)
        self.problem_type = self.featuretypes.pop(target_idx)

        logger.info("Init complete!")

    def predict(self, X: np.ndarray, feature_names: Iterable[str]) -> Union[np.ndarray, List, str, bytes]:
        """Returns a prediction.

        Args:
            X (numpy.array): Array-like with data.
            feature_names (iterable, optional): Array of feature names.

        Returns:
            Array-like with predictions.
        """
        logger.info("Predict called - will run predictions")

        # Builds a DataFrame from numpy.array.
        df = pd.DataFrame(X, columns=feature_names)

        # Encodes categorical features
        idx = 0
        for i, ft in enumerate(self.featuretypes):
            if ft == CATEGORICAL:
                x = df.iloc[:, i]
                df[df.columns[i]] = self.les[idx].transform(x)
                idx += 1

        # Replaces NaNs
        for i, col in enumerate(df.columns):
            ft = self.featuretypes[i]
            if ft == CATEGORICAL:
                sub = df[col].mode()[0]
                df[col].fillna(sub, inplace=True)
            elif ft == NUMERICAL:
                sub = df[col].mean()
                df[col].fillna(sub, inplace=True)

        # Performs Prediction
        X_predict = self.estimator.predict(df.to_numpy())

        # Applies Label Decoder for Classification Tasks
        if self.problem_type == CATEGORICAL:
            X_predict = self.les[-1].inverse_transform(X_predict)

        return X_predict

## Deployment Test

It simulates a model deployed by PlatIAgro

In [ ]:
%%sh
MODEL_NAME=Predictor
API_TYPE=REST
SERVICE_TYPE=MODEL
PERSISTENCE=0
LOG_LEVEL=DEBUG
PARAMETERS='[{"type":"STRING","name":"dataset","value":"iris"},{"type":"STRING","name":"target","value":"col4"},{"type":"STRING","name":"experiment_id","value":"99284308-cd3f-47d4-ab71-9c57acbb4d7b"}]'

seldon-core-microservice $MODEL_NAME $API_TYPE \
    --service-type $SERVICE_TYPE \
    --persistence $PERSISTENCE \
    --parameters $PARAMETERS \
    --log-level $LOG_LEVEL > log.txt 2>&1 &

## Make predictions

In [ ]:
!curl localhost:5000/predict -d 'json={"data":{"names":["col0", "col1", "col2", "col3"], "ndarray":[[5.1, 3.5, 1.4, 0.2]]}}'

## View logs

In [ ]:
!cat log.txt

## Cleans up the test

In [ ]:
!ps -ef | grep [s]eldon-core-microservice | awk '{print $2}' | xargs -r kill